# Load a Model, Test it on SNLI test set

In [ ]:
# using argparse:
!python train.py --mode test --optimizer adam --sent_encoder_model bilstmmax

In [ ]:
# using jupyter notebook:
from train import SentenceClassification

# set parameters
sent_encoder_model = "bilstmmax"
optimizer = "adam" # or "sgd"
trainer = SentenceClassification(sent_encoder_model=sent_encoder_model, epochs=max_epochs, seed=seed, resume_training=resume_training, verbose=False, optimizer_name=optimizer)

# testing
trainer.test_model()

## Inference Example

In [ ]:
output = trainer.infer("The cat is on the mat", "The cat is on the mat")

print(f'Predicted Class: {output["label"]}')
print(f'Scores: {output["output"]}')

# Results
